In [ ]:
pip install PyMuPDF tabula-py pandas

In [2]:
# import modules
import fitz  # PyMuPDF
import tabula
import pandas as pd

In [ ]:
def extract_images_from_pdf(pdf_path, image_folder):
    pdf_document = fitz.open(pdf_path)
    images = []

    for page_number in range(pdf_document.page_count):
        page = pdf_document[page_number]
        image_list = page.get_images(full=True)
        for img_index, img_info in enumerate(image_list):
            xref = img_info[0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_format = base_image["ext"]
            image_filename = f"{image_folder}/image_{page_number + 1}_{img_index + 1}.{image_format}"

            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)
            images.append(image_filename)

    pdf_document.close()
    return images

pdf_path = '/content/CDPHE OEPR Supply Chain Integrity Assessment Summary Report - FINAL.pdf'
image_folder = "images"
images = extract_images_from_pdf(pdf_path, image_folder)


In [ ]:
import tabula

def extract_tables_from_images(image_paths):
    tables = []

    for image_path in image_paths:
        table = tabula.read_pdf(image_path, pages="all", multiple_tables=True)
        tables.extend(table)

    return tables

tables = extract_tables_from_images(images)


In [ ]:
import pandas as pd

def save_tables_to_excel(tables, output_folder):
    for idx, table in enumerate(tables, start=1):
        excel_filename = f"{output_folder}/table_{idx}.xlsx"
        excel_writer = pd.ExcelWriter(excel_filename, engine="xlsxwriter")
        for table_idx, df in enumerate(table):
            df.to_excel(excel_writer, sheet_name=f"Sheet_{table_idx + 1}", index=False)
        excel_writer.save()

output_folder = "output_tables"
save_tables_to_excel(tables, output_folder)


# New Section